# Training the model on a Custom Dataset (BBC)

In [ ]:
import os

### Preprocessing

In [ ]:
#Converting each text document into a relevant format to pass for training

for filename in os.listdir("/content/drive/MyDrive/nlpyang/BBC/BBC News Summary/News Articles/business"):
  with open("/content/drive/MyDrive/nlpyang/BBC/BBC News Summary/Summaries/business/"+filename) as f1:
    with open("/content/drive/MyDrive/nlpyang/BBC/BBC News Summary/News Articles/business/"+filename,'a') as f2:
      for line in f1:
        lines = line.split('.')
        lines.sort(key = len)
        lines = lines[-3:]
        for summ in lines:
          f2.write("\n")
          f2.write("@highlight\n")
          f2.write("\n")
          f2.write(summ+"\n") 

In [ ]:
for filename in os.listdir("/content/drive/MyDrive/nlpyang/BBC/BBC News Summary/News Articles/entertainment"):
  with open("/content/drive/MyDrive/nlpyang/BBC/BBC News Summary/Summaries/entertainment/"+filename) as f1:
    with open("/content/drive/MyDrive/nlpyang/BBC/BBC News Summary/News Articles/entertainment/"+filename,'a') as f2:
      for line in f1:
        lines = line.split('.')
        lines.sort(key = len)
        lines = lines[-3:]
        for summ in lines:
          f2.write("\n")
          f2.write("@highlight\n")
          f2.write("\n")
          f2.write(summ+"\n") 

In [ ]:
for filename in os.listdir("/content/drive/MyDrive/nlpyang/BBC/BBC News Summary/News Articles/politics"):
  with open("/content/drive/MyDrive/nlpyang/BBC/BBC News Summary/Summaries/politics/"+filename) as f1:
    with open("/content/drive/MyDrive/nlpyang/BBC/BBC News Summary/News Articles/politics/"+filename,'a') as f2:
      for line in f1:
        lines = line.split('.')
        lines.sort(key = len)
        lines = lines[-3:]
        for summ in lines:
          f2.write("\n")
          f2.write("@highlight\n")
          f2.write("\n")
          f2.write(summ+"\n") 

In [ ]:
for filename in os.listdir("/content/drive/MyDrive/nlpyang/BBC/BBC News Summary/News Articles/sport"):
  with open("/content/drive/MyDrive/nlpyang/BBC/BBC News Summary/Summaries/sport/"+filename) as f1:
    with open("/content/drive/MyDrive/nlpyang/BBC/BBC News Summary/News Articles/sport/"+filename,'a') as f2:
      for line in f1:
        lines = line.split('.')
        lines.sort(key = len)
        lines = lines[-3:]
        for summ in lines:
          f2.write("\n")
          f2.write("@highlight\n")
          f2.write("\n")
          f2.write(summ+"\n") 

In [ ]:
for filename in os.listdir("/content/drive/MyDrive/nlpyang/BBC/BBC News Summary/News Articles/tech"):
  with open("/content/drive/MyDrive/nlpyang/BBC/BBC News Summary/Summaries/tech/"+filename) as f1:
    with open("/content/drive/MyDrive/nlpyang/BBC/BBC News Summary/News Articles/tech/"+filename,'a') as f2:
      for line in f1:
        lines = line.split('.')
        lines.sort(key = len)
        lines = lines[-3:]
        for summ in lines:
          f2.write("\n")
          f2.write("@highlight\n")
          f2.write("\n")
          f2.write(summ+"\n") 

In [ ]:
export CLASSPATH=/content/drive/MyDrive/nlpyang/BBC/BBC News Summary/stanford-corenlp-4.2.0/stanford-corenlp-4.2.0.jar

In [ ]:
!echo "Please tokenize this text." | java -cp /content/drive/MyDrive/nlpyang/stanford-corenlp-4.2.0/stanford-corenlp-4.2.0.jar edu.stanford.nlp.process.PTBTokenizer

In [ ]:
set CLASSPATH -cp /content/drive/MyDrive/nlpyang/stanford-corenlp-4.2.0/stanford-corenlp-4.2.0.jar

In [ ]:
import os
path = '/content/drive/MyDrive/nlpyang/PreSumm-master/custom_data'
files = os.listdir(path)


for index, file in enumerate(files):
  os.rename(os.path.join(path, file), os.path.join(path, ''.join([file[:-4], '.story'])))

In [ ]:
!python /content/drive/MyDrive/nlpyang/PreSumm-master/src/preprocess.py -mode tokenize -raw_path /content/drive/MyDrive/nlpyang/PreSumm-master/custom_data -save_path /content/drive/MyDrive/nlpyang/PreSumm-master/custom_tokenized

In [ ]:
li = os.listdir("/content/drive/MyDrive/nlpyang/PreSumm-master/custom_tokenized")
len(li)

In [ ]:
!python /content/drive/MyDrive/nlpyang/PreSumm-master/src/preprocess.py -mode format_to_lines -raw_path /content/drive/MyDrive/nlpyang/PreSumm-master/custom_tokenized -save_path /content/drive/MyDrive/nlpyang/PreSumm-master/json_data/cnndm -n_cpus 1 -use_bert_basic_tokenizer false

In [ ]:
!python /content/drive/MyDrive/nlpyang/PreSumm-master/src/preprocess.py -mode format_to_bert -raw_path /content/drive/MyDrive/nlpyang/PreSumm-master/json_data -save_path /content/drive/MyDrive/nlpyang/PreSumm-master/Step5  -lower -n_cpus 1 -log_file /content/drive/MyDrive/nlpyang/PreSumm-master/logs/preprocess.log

In [ ]:
#Adding a [CLS][SEP] between all consecutive lines in a document as a part of preprocessing

with open("/content/drive/MyDrive/summarization/raw input/navigation_amazon_kindle.txt.data") as f1:
    with open("/content/drive/MyDrive/summarization/raw input/navigation_amazon_kindle_ext2.txt.data",'w') as f2:
      for line in f1:
        lines = line.split('.')[0]
        f2.write(" " + "[CLS]" + " " + "[SEP]" + " " + lines + ".")

In [ ]:
print(lines)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install pytorch_transformers

In [ ]:
pip install tensorboardX

In [ ]:
pip install torch==1.1.0

In [ ]:
pip install pyrouge

In [ ]:
!python /content/drive/MyDrive/nlpyang/PreSumm-master/src/train.py -task ext -mode train -bert_data_path /content/drive/MyDrive/nlpyang/PreSumm-master/Step5/cnndm -ext_dropout 0.1 -model_path /content/drive/MyDrive/nlpyang/PreSumm-master/CUSTOM_MODEL -lr 2e-3 -visible_gpus 0 -report_every 50 -save_checkpoint_steps 1000 -batch_size 300 -train_steps 10000 -accum_count 5 -log_file /content/drive/MyDrive/nlpyang/PreSumm-master/logs/custom -use_interval true -warmup_steps 10000 -max_pos 512

In [ ]:
!python /content/drive/MyDrive/nlpyang/PreSumm-dev/src/train.py -task ext -mode test_text -text_src /content/drive/MyDrive/nlpyang/PreSumm-dev/raw_data/temp_ext.raw_src -result_path /content/drive/MyDrive/nlpyang/PreSumm-dev/raw_data/output_2.txt -test_from /content/drive/MyDrive/nlpyang/PreSumm-master/MODEL_PATH/model_step_10000.pt

In [ ]:
!python /content/drive/MyDrive/nlpyang/PreSumm-master/src/train.py -task ext -mode test -batch_size 3000 -test_batch_size 50 -bert_data_path /content/drive/MyDrive/nlpyang/PreSumm-master/bert_data/_test/cnndm -log_file /logs/val_abs_bert_cnndm -model_path /content/drive/MyDrive/nlpyang/PreSumm-master/MODEL_PATH -test_from /content/drive/MyDrive/nlpyang/PreSumm-master/MODEL_PATH/model_step_10000.pt -sep_optim true -use_interval true -visible_gpus 0 -max_pos 512 -max_length 200 -alpha 0.95 -min_length 50 -result_path /logs/abs_bert_cnndm 


In [ ]:
pip install tensorboard